In [144]:
import urllib.request
import xml.etree.ElementTree as ET
from urllib.parse import urlencode, quote_plus

import time
import pandas as pd

url = 'http://data.ex.co.kr/openapi/vdsinfo/vdsList?'
serviceKey = 'key=8784419864'
m_page = 1
# 1 : 0010CZE010  1~7589
#range(7589)
#i+1
ids=[]
ways=[]
xx=[]
yy=[]
rts=[]
rtnos=[]

start =time.time()
for i in range(m_page):
    m_page=i+1
    queryParams = urlencode({quote_plus('type') : 'xml',
                             quote_plus('numOfRows') : '1',
                             quote_plus('pageNo') : '%d'%m_page })
    request = urllib.request.Request(url + serviceKey+'&'+ queryParams)
    request.get_method = lambda: 'GET'
    response_body = urllib.request.urlopen(request).read().decode("utf-8")
    tree = ET.fromstring(response_body)
    lst = tree.findall('list')

    for item in lst:
        
        ids.append(item.find('czId').text)
        ways.append(item.find('directionCode').text)
        xx.append(item.find('grs80x').text)
        yy.append(item.find('grs80y').text)
        rts.append(item.find('routeName').text)
        rtnos.append(item.find('routeNo').text)
end =time.time()

print('%d개 -> %d초 걸림'%(m_page,int(end-start)))
print('7589개 -> %d초 예상'%(7589*m_page/int(end-start)))
#10개 -> 4초 걸림
#7589개 -> 18972초 예상

100개 -> 45초 걸림
7589개 -> 16864초 예상


In [145]:
16864/3600

4.684444444444445

In [151]:
#import pandas as pd 
import numpy as np
df1 = pd.DataFrame()
a=[]
for i in range(len(ids)):
    tmp = [ids[i],ways[i],xx[i],yy[i],rtnos[i],rts[i]]
    a.append(tmp)
   
df1 = df1.append(a)

df1.columns=['콘존ID','방향','grs80x','grs80y','노선번호','노선명']
df1.head(5)

,콘존ID,방향,grs80x,grs80y,노선번호,노선명
0,0010CZE010,E,391265.163647,297630.026461,0010,경부선
1,0010CZE011,E,391642.593415,298552.164856,0010,경부선
2,0010CZE011,E,391873.608131,299421.834737,0010,경부선
3,0010CZE020,E,391448.581180,300529.001930,0010,경부선
4,0010CZE030,E,391067.929665,300978.024563,0010,경부선


In [162]:
len(df1[df1['노선명']=='경부선'])

100

In [161]:
from pyproj import Proj, transform
import numpy as np
import pandas  as pd

# Projection 정의
# UTM-K
proj_UTMK = Proj(init='epsg:5178') # UTM-K(Bassel) 도로명주소 지도 사용 중

# WGS1984
proj_WGS84 = Proj(init='epsg:4326') # Wgs84 경도/위도, GPS사용 전지구 좌표

# UTM-K -> WGS84 샘플
x1, y1 = 961114.519726,1727112.269174
x2, y2 = transform(proj_UTMK,proj_WGS84,x1,y1)
print(x2,y2)

# WGS84 -> UTM-K 샘플
x1, y1 = 127.07098392510115, 35.53895289091983
x2, y2 = transform(proj_WGS84, proj_UTMK, x1, y1)
print(x2,y2)

# x, y 컬럼을 이용하여 UTM-K좌표를 WGS84로 변환한 Series데이터 반환
def transform_utmk_to_w84(df):
    return pd.Series(transform(proj_UTMK, proj_WGS84, df['x'], df['y']), index=['x', 'y'])

df_xy = pd.DataFrame([
                                        ['A', xx[0],yy[0]],
                                        ['B',  xx[1],yy[1]],
                                        ['C',  xx[2],yy[2]]
                                    ], columns=['id', 'x', 'y'])

df_xy[['x_w84', 'y_w84']] = df_xy.apply(transform_utmk_to_w84, axis=1)


127.0688934553203 35.54198138473581
961302.5606361036 1726775.5570293881


In [ ]:
        '''
        print('콘존ID:',item.find('czId').text)
        print('기점종점방향구분코드:',item.find('directionCode').text)
        print('장비소속구분코드:',item.find('equipmentBelongingCode').text)
        print('장비소속구분명:',item.find('equipmentBelongingName').text)
        print('GRS80X좌표값:',item.find('grs80x').text)
        print('GRS80Y좌표값:',item.find('grs80y').text)
        print('도로등급구분코드:',item.find('roadgradeCode').text)
        print('roadgradeName:',item.find('roadgradeName').text)
        print('routeName:',item.find('routeName').text)
        print('routeNo:',item.find('routeNo').text)
        print('노선구성순번:',item.find('routeSeq').text)
        print('지점이정:',item.find('shift').text)
        print('VDS존유형구분코드:',item.find('vdsCode').text)
        print('VDS존종료이정:',item.find('vdsEndShift').text)
        print('VDS존시작이정:',item.find('vdsStartShift').text)
        print('vdsId:',item.find('vdsId').text)
        print('VDS존길이:',item.find('vdsLength').text)
        print('VDS존유형구분명:',item.find('vdsName').text)
        print('-----------------------------')
        '''

In [48]:
import urllib.request
import xml.etree.ElementTree as ET
from urllib.parse import urlencode, quote_plus

url = 'http://data.ex.co.kr/openapi/vdsinfo/vdsList?'
serviceKey = 'key=8784419864'
queryParams = urlencode({quote_plus('type') : 'xml',
                         quote_plus('numOfRows') : '1',
                         quote_plus('pageNo') : '120' })
request = urllib.request.Request(url + serviceKey+'&'+ queryParams)
request.get_method = lambda: 'GET'
response_body = urllib.request.urlopen(request).read().decode("utf-8")
#print(response_body)

tree = ET.fromstring(response_body)
print("---------------------------------------")
lst = tree.findall('list')
print('The nuumber of list:', len(lst))
print("---------------------------------------")

for item in lst:
    print('콘존ID:',item.find('czId').text)
    print('기점종점방향구분코드:',item.find('directionCode').text)
    print('장비소속구분코드:',item.find('equipmentBelongingCode').text)
    print('장비소속구분명:',item.find('equipmentBelongingName').text)
    print('GRS80X좌표값:',item.find('grs80x').text)
    print('GRS80Y좌표값:',item.find('grs80y').text)
    print('도로등급구분코드:',item.find('roadgradeCode').text)
    print('roadgradeName:',item.find('roadgradeName').text)
    print('routeName:',item.find('routeName').text)
    print('routeNo:',item.find('routeNo').text)
    print('노선구성순번:',item.find('routeSeq').text)
    print('지점이정:',item.find('shift').text)
    print('VDS존유형구분코드:',item.find('vdsCode').text)
    print('vdsId:',item.find('vdsId').text)
    print('VDS존길이:',item.find('vdsLength').text)
    print('VDS존종료이정:',item.find('vdsEndShift').text)
    print('VDS존시작이정:',item.find('vdsStartShift').text)
    print('VDS존유형구분명:',item.find('vdsName').text)
    print('-----------------------------')
    
    #콘존ID: 0010CZE130
#기점종점방향구분코드: E

---------------------------------------
The nuumber of list: 1
---------------------------------------
콘존ID: 0010CZE130
기점종점방향구분코드: E
장비소속구분코드: 00
장비소속구분명: 본선
GRS80X좌표값: 351192.277797
GRS80Y좌표값: 368364.804976
도로등급구분코드: 101
roadgradeName: 고속국도
routeName: 경부선
routeNo: 0010
노선구성순번: 120
지점이정: 124.60km
VDS존유형구분코드: 1
vdsId: 0010VDE07200
VDS존길이: 1000.00m
VDS존종료이정: 125.10km
VDS존시작이정: 124.10km
VDS존유형구분명: FTMS 본선 VDS
-----------------------------
